TSG116 - SQL BDC security support log translator
================================================

Get logs
--------

This notebook is designed to parse the logs generated by the security
support service in SQL BDC. To get the logs, we will copy the debug logs
from the cluster and extract them. Follow the steps below - run “azdata
bdc debug copy-logs -n *\<namespace\>*”. This will create several
.tar.gz files - Extract the contents of
debuglogs-*\<namespace\>*-*\<date\>*-*\<time\>*.tar.gz - Locate the
security support log stored at
./*\<namespace\>*/control-*\<…\>*/security-support/supervisol/log/secsupp-stderr---*\<…\>*.log

### Parameters

In [ ]:
filePath = ""

Translate error messages
------------------------

Run the code block below and pass the path the the security support log
located in the previous step

In [ ]:
# Parse log files
#
import re
from IPython.display import display, Markdown

errorDict = {
    "Local error":
        "DNS resolution failed. Ensure forward and reverse DNS queries for the domain (e.g. contoso.com)"
        " work on the cluster and the correct values are specified for domainControllerFullyQualifiedDns"
        " in control.json before cluster deployment.",
    "No such object":
        "Unable to find the requested object in Active Directory. Check that the OU specified in"
        " control.json exists.",
    "Insufficient access":
        "The AD account provided to BDC deployment does not have the necessary permissions. Check"
        " [here](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15)"
        " for instructions on setting the appropriate permissions",
    "Access denied":
        "The AD account provided to BDC deployment does not have the necessary permissions. Check"
        " [here](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15)"
        " for instructions on setting the appropriate permissions",
    "Realm not local to KDC":
        "Mismatched domain and KDC. The KDC and domain provided in control.json during deployment are not"
        " part of the same domain. Check dnsIpAddresses, domainControllerFullyQualifiedDns, and domainDnsName"
        " in control.json",
    "Unknown API error":
        "Unable to authenticate to server. Ensure that username and password for the AD deployment account are"
        " correct and that domain information in control.json is valid (domainDnsName, etc).",
    "Cannot contact any KDC for realm '[^ ']+'":
        "No input domain controllers responded to AD requests. Check dnsIpAddresses in control.json is a"
        " valid domain controller.",
    "Preauthentication failed":
        "Invalid password. Check the password supplied for the AD deployment account is correct.",
    "Client '[^ ']+@[^ ']+' not found in Kerberos database":
        "Invalid user. Check the username supplied for the AD deployment account is correct.",
}

def FindErrorCause(errorString) :
    res = None

    for knownErrorRegex, cause in errorDict.items():
        if re.search(knownErrorRegex, errorString):
            res = cause
            break

    return res

def ParseLogFile() :
    foundError = False

    with open(filePath, 'r') as fp:
        lineNum = 0

        while True:
            line = fp.readline()
            lineNum += 1

            if not line:
                break

            # Strip off time stamps which are always the first 19 characters (YYYY/MM/DD HH:MM:SS)
            #
            line = line[19:].strip()

            # Find if there is an error
            #
            errorMatch = re.search('Failed.*:', line)

            if errorMatch:
                internalErrorString = line[errorMatch.end() + 1:].rstrip()
                cause = FindErrorCause(internalErrorString)

                if cause:
                    display(Markdown("Line {0}: {1}  \n{2}".format(lineNum, line, cause)))
                    foundError = True

    if not foundError:
        print("No translatable errors found in the log.")

print("Log file location:")

if filePath == "":
    filePath = input().strip('"')
else:
    print("")

print(filePath)
ParseLogFile()

Next steps
----------

If no translated error messages were found, check the following: - No
accounts still exist from a previous BDC Deployment on this domain. Them
being in a different OU can still cause problems. - In control.json,
make sure that dnsIpAddresses and domainControllerFullyQualifiedDns
point to the same host. Them pointing to two different domain
controllers can cause issues. - Don’t use “Domain Users” group. Some
services that SQL Server BDC relies on do not recognize “Doamin Users”
as a valid group.

For more troubleshooting and help with AD deployment, [visit the
docs](https://docs.microsoft.com/en-us/sql/big-data-cluster/deploy-active-directory?view=sql-server-ver15).

In [ ]:
print("Notebook execution is complete.")